In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
headers = {
    # 'Host': 'i.meizitu.net',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
    'Connection': 'keep-alive',
    'Accept-Encoding':'gzip, deflate, br'#,
#     'Upgrade-Insecure-Requests':'1'
}

In [3]:
s = requests.session()

In [4]:
url_re = s.get('https://www.msn.com/', headers=headers)

In [5]:
url_re

<Response [200]>

In [33]:
df = []

In [6]:
soup = BeautifulSoup(url_re.text)

In [30]:
soup.select('img[alt="Business Insider Logo"]')

[<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=png"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif"/>]

In [ ]:
soup

In [116]:
for index, item in enumerate(soup.select('.stripecontainer[aria-label]'), start = 2):

        print(item['aria-label'])


new york
news
from our partners
entertainment
sports
money
lifestyle
autos
health & fitness
food & drink
travel
causes
video
promotions


In [101]:
ib.attrs

{'class': ['tertiary'],
 'data-aop': 'tertiary',
 'data-id': '621',
 'data-m': '{"i":621,"p":618,"n":"tertiary","y":8,"o":2}'}

In [109]:
re.findall("\"o\":(\d+)",ib['data-m'])[0]

'2'

In [110]:
ibl

<li data-id="624" data-m='{"i":624,"p":621,"n":"HeadlineItemViewModel","y":13,"l":"AAGh0FD","h":0,"v":"foodanddrink","c":"foodnews","o":3}'>
<a href="/en-us/foodanddrink/foodnews/the-ceo-of-popeyes-parent-company-breaks-down-the-massive-success-of-its-chicken-sandwich/ar-AAGh0FD?li=BBnb7Kw">
<img alt="" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img?h=75&amp;w=100&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;x=902&amp;y=1027"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif" title="The CEO of Popeyes' parent company breaks down the massive success of its chicke... - Irene Jiang / Business Insider"/> <div>
<h3 aria-label="popeyes' parent company ceo on chicken sandwich success-article">Popeyes' parent company CEO on chicken sandwich success</h3>
<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp

In [111]:
ibl.attrs

{'data-id': '624',
 'data-m': '{"i":624,"p":621,"n":"HeadlineItemViewModel","y":13,"l":"AAGh0FD","h":0,"v":"foodanddrink","c":"foodnews","o":3}'}

In [115]:
df = []
for index, item in enumerate(soup.select('.stripecontainer[aria-label]'), start = 2):
    if item.select('img[alt="Business Insider Logo"]'):
#         print(item)
        print(index)
        river_position = index
        river_name = item['aria-label']
        a = item

        for item_block in item.find_all('ul'):
            if item_block.select('img[alt="Business Insider Logo"]'):
                ib = item_block
                item_block_type = item_block['data-aop']
                item_block_position = re.findall("\"o\":(\d+)",item_block['data-m'])[0]
#                 print(item_block)
                for item_block_link in item_block.select('li'):
                    if item_block_link.select('img[alt="Business Insider Logo"]'):
                        ibl = item_block_link
                        x = item_block_link
                        print(item_block_link)
                        print(len(item_block_link))
                        item_block_link_position = re.findall("\"o\":(\d+)",item_block_link['data-m'])[0]
                        artile_tile = item_block_link.select('h3[aria-label]')[0]['aria-label']
                        artile_img = re.findall(r'\/\/(.*)\?',item_block_link.select('img[alt=""]')[0]['data-src'])[0]
                        df.append([river_position, 
                                   river_name,
                                   item_block_position,
                                   item_block_type,
                                   item_block_link_position,
                                   artile_tile,
                                   artile_img])
        
        
    else:
        print('***')
pd.DataFrame(df, columns = ['river_position',
                           'river_name',
                           'item_block_position',
                           'item_block_type',
                           'item_block_link_position',
                           'artile_title',
                           'artile_img'])

***
***
***
***
***
***
***
***
***
11
<li data-id="624" data-m='{"i":624,"p":621,"n":"HeadlineItemViewModel","y":13,"l":"AAGh0FD","h":0,"v":"foodanddrink","c":"foodnews","o":3}'>
<a href="/en-us/foodanddrink/foodnews/the-ceo-of-popeyes-parent-company-breaks-down-the-massive-success-of-its-chicken-sandwich/ar-AAGh0FD?li=BBnb7Kw">
<img alt="" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img?h=75&amp;w=100&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;x=902&amp;y=1027"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif" title="The CEO of Popeyes' parent company breaks down the massive success of its chicke... - Irene Jiang / Business Insider"/> <div>
<h3 aria-label="popeyes' parent company ceo on chicken sandwich success-article">Popeyes' parent company CEO on chicken sandwich success</h3>
<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityi

,river_position,river_name,item_block_position,item_block_type,item_block_link_position,artile_title,artile_img
0,11,food & drink,2,tertiary,3,popeyes' parent company ceo on chicken sandwic...,static-global-s-msn-com.akamaized.net/img-resi...


In [77]:
x

<li data-id="624" data-m='{"i":624,"p":621,"n":"HeadlineItemViewModel","y":13,"l":"AAGh0FD","h":0,"v":"foodanddrink","c":"foodnews","o":3}'>
<a href="/en-us/foodanddrink/foodnews/the-ceo-of-popeyes-parent-company-breaks-down-the-massive-success-of-its-chicken-sandwich/ar-AAGh0FD?li=BBnb7Kw">
<img alt="" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img?h=75&amp;w=100&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;x=902&amp;y=1027"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif" title="The CEO of Popeyes' parent company breaks down the massive success of its chicke... - Irene Jiang / Business Insider"/> <div>
<h3 aria-label="popeyes' parent company ceo on chicken sandwich success-article">Popeyes' parent company CEO on chicken sandwich success</h3>
<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp

In [82]:
x.select('h3[aria-label]')[0]['aria-label']

"popeyes' parent company ceo on chicken sandwich success-article"

In [97]:
re.findall(r'\/\/(.*)\?',x.select('img[alt=""]')[0]['data-src'])[0]

'static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img'

In [92]:
x.select('img[alt=""]')[0]['data-src']

'{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img?h=75&w=100&m=6&q=60&u=t&o=t&l=f&x=902&y=1027"}'

In [57]:
a.select('.stripecontainer[aria-label]')

[]

In [63]:
a.find_all('ul')

[<ul>
 <li data-id="604" data-m='{"i":604,"p":603,"n":"foodanddrink","y":4,"o":1}'>
 <h2><a data-id="605" data-m='{"i":605,"p":603,"n":"foodanddrink","y":4,"o":2}' href="/en-us/foodanddrink">Food &amp; Drink</a></h2>
 </li>
 <li class="hide12" data-id="606" data-m='{"i":606,"p":603,"n":"Food News","y":4,"o":3}'><a href="/en-us/foodanddrink/restaurantsandnews">FOOD NEWS</a></li>
 <li class="hide12" data-id="607" data-m='{"i":607,"p":603,"n":"Restaurants","y":4,"o":4}'><a href="/en-us/foodanddrink/restaurantsandnews/casual">RESTAURANTS</a></li>
 <li class="hide12" data-id="608" data-m='{"i":608,"p":603,"n":"Recipes","y":4,"o":5}'><a href="/en-us/foodanddrink/cooking/recipes">RECIPES</a></li>
 <li class="hide12" data-id="609" data-m='{"i":609,"p":603,"n":"Quick \u0026 Easy","y":4,"o":6}'><a href="/en-us/foodanddrink/cooking/quickandeasy">QUICK &amp; EASY</a></li>
 <li class="hide12" data-id="610" data-m='{"i":610,"p":603,"n":"Tips \u0026 Tricks","y":4,"o":7}'><a href="/en-us/foodanddrink/

In [62]:
a

<div aria-label="food &amp; drink" class="stripecontainer" data-section-id="stripe.foodanddrink" role="region">
<div class="fooddrink stripeouter" data-aop="stripe.foodanddrink_stripe" data-id="601" data-m='{"i":601,"p":25,"n":"stripe.foodanddrink","t":"stripe","o":14}'>
<button class="leftarrow" data-id="602" data-m='{"i":602,"p":601,"n":"leftarrow","y":12,"o":1}' tabindex="-1" title="previous"></button>
<div class="stripe full-width">
<div class="stripenav" data-aop="stripe.foodanddrink.navigation_stripenavigation" data-id="603" data-m='{"i":603,"p":601,"n":"stripe.foodanddrink.navigation","t":"stripeNavigation","o":2}'>
<ul>
<li data-id="604" data-m='{"i":604,"p":603,"n":"foodanddrink","y":4,"o":1}'>
<h2><a data-id="605" data-m='{"i":605,"p":603,"n":"foodanddrink","y":4,"o":2}' href="/en-us/foodanddrink">Food &amp; Drink</a></h2>
</li>
<li class="hide12" data-id="606" data-m='{"i":606,"p":603,"n":"Food News","y":4,"o":3}'><a href="/en-us/foodanddrink/restaurantsandnews">FOOD NEWS</a

In [59]:
a.attrs

{'class': ['stripecontainer'],
 'role': 'region',
 'aria-label': 'food & drink',
 'data-section-id': 'stripe.foodanddrink'}

In [61]:
a.select('img[alt="Business Insider Logo"]')

[<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=png"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif"/>]

In [38]:
df

[[11, 'food & drink'], {11, 'food & drink'}]

In [39]:
pd.DataFrame(df)

,0,1
0,11,food & drink
1,food & drink,11


In [44]:
soup.select('.stripecontainer[aria-label]')[9].select('img[alt="Business Insider Logo"]')

[<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=png"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif"/>]